In [16]:
import pandas as pd 
import numpy as np 
import pyarrow as pa
import pyarrow.parquet as pq
import time
import os
from scipy.optimize import root_scalar

In [8]:

# A function to compute minor allele frequency
def eaf_to_maf(eaf):
    if eaf<=0.5:
        return eaf
    else:
        return 1-eaf


# </br> Merging metabolite_1 and metabolite_2 summary statistics files:<br>


In [42]:
met2_file="../metabolite_sumstats/GCST90200425_harmonized.parquet.gz"
met1_file="../metabolite_sumstats/GCST90199752_harmonized.parquet.gz"


met1 = pd.read_parquet(met1_file)
met2 = pd.read_parquet(met2_file)
gwas_catalog_id_1 = met1_file.split('/')[-1].split('.')[0].split('_')[0]
gwas_catalog_id_2 = met2_file.split('/')[-1].split('.')[0].split('_')[0]
# Reading metabolite ratio sum stats file

ratio_sumstat = pd.merge(met1, met2, suffixes=['_1', '_2'], how='inner',
                           on=['position', 'chromosome', 'effect_allele', 'reference_allele', 'pos_name',
                               'variant_id'
                               ])  # Merging with the ratio on position of SNPs
maf_threshold=0.01

#        pval_threshold=1e-5
ratio_sumstat['maf'] = ratio_sumstat['effect_allele_frequency_1'].apply(eaf_to_maf)
ratio_sumstat = ratio_sumstat[ratio_sumstat.maf > maf_threshold]

ratio_sumstat['z_1'] = ratio_sumstat['beta_1'] / ratio_sumstat['se_1']
ratio_sumstat['z_2'] = ratio_sumstat['beta_2'] / ratio_sumstat['se_2']



# </br>Obtaining The Correlation Term:<br>
## Choosing A correlation term that would make the lambda (genomic inflation factor) equal to 1.

</br>ratio_sumstat should have the following columns at least: <br>
</br>beta_1 : effect size of the numerator summary statistics file
</br>beta_2 : effect size of the denominator summary statistics file
</br>se_1   : standard error of the numerator summary statistics file
</br>se_2   : standard error of the denominator summary statistics file


In [43]:
def objective(correlation, df):
    """Objective function for root_scalar, now accepting a dataframe."""
    beta_built_ratio = df['beta_1'] - df['beta_2']
    se_built_ratio = df['se_1']**2 + df['se_2']**2 - 2 * correlation * df['se_1'] * df['se_2']
    se_built_ratio[se_built_ratio < 0] = 0 # Handle potential floating point errors
    se_built_ratio = np.sqrt(se_built_ratio)
    
    z_built_ratio = np.divide(beta_built_ratio, se_built_ratio, 
                              out=np.zeros_like(beta_built_ratio), 
                              where=se_built_ratio!=0)

    lamb = np.median(z_built_ratio**2) / 0.454936423119572
    return lamb - 1


In [44]:

result = root_scalar(objective, args=(ratio_sumstat,), bracket=[-1, 1], method='brentq')
correlation = result.root


ratio_sumstat['z_1'] = ratio_sumstat['beta_1'] / ratio_sumstat['se_1']
ratio_sumstat['z_2'] = ratio_sumstat['beta_2'] / ratio_sumstat['se_2']

ratio_sumstat = ratio_sumstat.loc[ratio_sumstat.beta_1 * ratio_sumstat.beta_2 < 0]

se_calc = np.sqrt(ratio_sumstat['se_1']**2 + ratio_sumstat['se_2']**2 - 2 * correlation * ratio_sumstat['se_1'] * ratio_sumstat['se_2'])
ratio_sumstat['se'] = se_calc
ratio_sumstat['beta'] = ratio_sumstat['beta_1'] - ratio_sumstat['beta_2']
ratio_sumstat['z'] = ratio_sumstat['beta'] / ratio_sumstat['se']

